In [ ]:
### Import packages 
import networkx as nx # for handling graphs/networks 
import numpy as np # for basic scientific computing 
import pandas as pd # for basic scientific computing 
import matplotlib.pyplot as plt # for plotting
import matplotlib.gridspec as gridspec

import seaborn as sns

from scipy.special import binom
from scipy.linalg import subspace_angles

from scipy.integrate import solve_ivp
import Koopman_source as kp
from math import comb
#from copy import copy
import csv  
from scipy.optimize import minimize
from scipy.special import expit


In [ ]:
np.random.seed(123)

## Compute a Koopman matrix from simulated data.

#x0 = [2,1]

model = "Duffing"
sequential = False

if model == "d2_ans":
    d = 2
elif model == "d2":
    d = 2 # the number of variables (=the dimension) of the original dynamical system.
elif model == "d3":
    d = 3
elif model == "Lorenz":
    d = 3
elif model == "Lotka_Voltera":
    d = 2
elif model == "Duffing":
    d = 2

#M = 100 # the number of observed data. 

orth_basis = True

M_max = 2000 # the maximum number of observed data. 
n_repeat = 2 # the number of repetition of approximaing the Koopman matrix. 
max_order = 6
N = comb(d+max_order,d) # the total number of observable functions, psi_m's. 
# N1 = 4 
# N2 = N - N1

In [ ]:

K_approx_accum = np.zeros([N,N,M_max,n_repeat])
G_approx_accum = np.zeros([N,N,M_max,n_repeat])
G_pinv_accum = np.zeros([N,N,M_max,n_repeat])
A_approx_accum = np.zeros([N,N,M_max,n_repeat])
P_accum = np.zeros([M_max,M_max,M_max,n_repeat])


alpha0 = -1
beta0 = 1
delta0 = 0.5 # Damping parameter
dt = 0.02

for rr in range(n_repeat):
    if np.mod(rr, 2) == 0:
        print(rr)
    x_mat_total = np.random.default_rng().exponential(scale=1, size=[d,M_max])
    # x_mat_total = np.zeros([d,M_max])
    # x_mat_total[:,0] = np.array([0.1,0.1])
    y_mat_total = np.zeros([d,M_max])

    K_approx = np.zeros([N,N,M_max])
    G_approx = np.zeros([N,N,M_max])
    G_pinv = np.zeros([N,N,M_max])
    A_approx = np.zeros([N,N,M_max])

    for i in range(M_max):
        if sequential == True:
            if model == "Duffing":
                y_mat_total[:,i] = kp.Duffing(x_mat_total[:,i], alpha0=alpha0, beta0 = beta0, delta0 = delta0, dt = dt)
                if i+1 < M_max:
                    x_mat_total[:,i+1] = y_mat_total[:,i]
        else:
            if model == "Duffing":
                y_mat_total[:,i] = kp.Duffing(x_mat_total[:,i], alpha0=alpha0, beta0 = beta0, delta0 = delta0, dt = dt)

    Psi_X = np.zeros([M_max,N])
    Psi_Y = np.zeros([M_max,N])

    for i in range(M_max):
        if model == "Duffing":
            Psi_X[i,:], ind_mat = kp.psi_Laguerre(x_mat_total[:,i], d, max_order)
            Psi_Y[i,:] = kp.psi_Laguerre(y_mat_total[:,i], d, max_order)[0]

    G_approx[:,:,0] = np.matmul(Psi_X[np.ix_(range(1),range(N))].transpose(), Psi_X[np.ix_(range(1),range(N))])
    G_pinv[:,:,0] = np.linalg.pinv(G_approx[:,:,0])
    A_approx[:,:,0] = np.matmul(Psi_X[np.ix_(range(1),range(N))].transpose(), Psi_Y[np.ix_(range(1),range(N))])
    K_approx[:,:,0] = np.matmul(G_pinv[:,:,0], A_approx[:,:,0])

    for M in range(2,M_max+1):  
        G_approx[:,:,M-1] = G_approx[:,:,M-2] + np.matmul(Psi_X[np.ix_(range(M-1,M),range(N))].transpose(), Psi_X[np.ix_(range(M-1,M),range(N))])
        G_pinv[:,:,M-1] = np.linalg.pinv(G_approx[:,:,M-1])
        A_approx[:,:,M-1] = A_approx[:,:,M-2] + np.matmul(Psi_X[np.ix_(range(M-1,M),range(N))].transpose(), Psi_Y[np.ix_(range(M-1,M),range(N))])
        K_approx[:,:,M-1] = np.matmul(G_pinv[:,:,M-1], A_approx[:,:,M-1])

    G_approx_accum[:,:,:,rr] = G_approx
    G_pinv_accum[:,:,:,rr] = G_pinv
    A_approx_accum[:,:,:,rr] = A_approx
    K_approx_accum[:,:,:,rr] = K_approx

In [ ]:
mixed_idx = np.random.permutation(N)

data = K_approx[:,:,M_max-1]
data = 1/dt * (np.squeeze(data) - np.identity(N))

eig_v = kp.pagerank(abs(data), 10000, 0.85)
eig_v2 = kp.subsampled_pagerank(abs(data), 10000, 0.85, 0.2)
ranked_idx = np.argsort(eig_v)[::-1]
ranked_idx2 = np.argsort(eig_v2)[::-1]

data_mx = data[np.ix_(mixed_idx,mixed_idx)]
# data_mx = 1/dt * (np.squeeze(data_mx) - np.identity(N))

data_pr = data[np.ix_(ranked_idx,ranked_idx)]
# data_pr = 1/dt * (np.squeeze(data_pr) - np.identity(N))

data_subpr = data[np.ix_(ranked_idx2,ranked_idx2)]
# data_subpr = 1/dt * (np.squeeze(data_subpr) - np.identity(N))

# Create a heatmap
#fig2, ax2 = plt.subplots(len(i_list), len(j_list), figsize = (8,8))
fig1, ax1 = plt.subplots(1,3,figsize = (15,4))
#plt.figure(figsize=(8, 6))
sns.heatmap(expit(data*1), ax=ax1[0], annot=True, cmap='coolwarm', cbar=True)
sns.heatmap(expit(data_mx*1), ax=ax1[1], annot=True, cmap='coolwarm', cbar=True)
sns.heatmap(expit(data_pr*1), ax=ax1[2], annot=True, cmap='coolwarm', cbar=True)# sns.heatmap(expit(data_subpr*10), ax=ax1[2], annot=True, cmap='coolwarm', cbar=True)
# sns.heatmap(expit(data_pr*100), annot=True, cmap='coolwarm', cbar=True)
# sns.heatmap(1/(1+np.exp(-data)), annot=True, cmap='coolwarm', cbar=True)

# Add title and labels
plt.title("Heatmap Example")
plt.xlabel("Columns")
plt.ylabel("Rows")


In [ ]:
# Computing the subspace angles with increasing number of (ordered) basis functions.

subsp_ang_mx = np.zeros(N)
subsp_ang_inc = np.zeros(N)
subsp_ang_pr = np.zeros(N)
subsp_ang_subpr = np.zeros(N)

M_ang = 1000
dataX_for_angle = np.random.default_rng().exponential(scale=1, size=[d,M_ang])
dataY_for_angle = np.random.default_rng().exponential(scale=1, size=[d,M_ang])
Psi_X_ang = np.zeros([M_ang,N])
Psi_Y_ang = np.zeros([M_ang,N])

for i in range(M_ang):
    dataY_for_angle[:,i] = kp.Duffing(dataX_for_angle[:,i], alpha0=alpha0, beta0 = beta0, delta0 = delta0, dt = dt)

for i in range(M_ang):
    Psi_X_ang[i,:], ind_mat = kp.psi_Laguerre(dataX_for_angle[:,i], d, max_order)
    Psi_Y_ang[i,:] = kp.psi_Laguerre(dataY_for_angle[:,i], d, max_order)[0]
    
for N_vary in range(2,N):
    # x_mat_total = np.random.default_rng().exponential(scale=1, size=[d,M_max])
    # Psi_Y_ang = np.matmul(Psi_X_ang[np.ix_(range(M_ang),range(N_vary))], data[np.ix_(range(N_vary),range(N_vary))])
    subsp_ang_inc[N_vary] = subspace_angles(Psi_X_ang[np.ix_(range(M_ang),range(N_vary))], Psi_Y_ang[np.ix_(range(M_ang),range(N_vary))])[0]
    subsp_ang_mx[N_vary] = subspace_angles(Psi_X_ang[np.ix_(range(M_ang),mixed_idx[range(N_vary)])], Psi_Y_ang[np.ix_(range(M_ang),mixed_idx[range(N_vary)])])[0]
    subsp_ang_pr[N_vary] = subspace_angles(Psi_X_ang[np.ix_(range(M_ang),ranked_idx[range(N_vary)])], Psi_Y_ang[np.ix_(range(M_ang),ranked_idx[range(N_vary)])])[0]
    # subsp_ang_subpr[N_vary] = subspace_angles(Psi_X_ang[np.ix_(range(M_ang),ranked_idx2[range(N_vary)])], Psi_Y_ang[np.ix_(range(M_ang),ranked_idx2[range(N_vary)])])[0]



In [ ]:
plt.plot(subsp_ang_inc * 180/np.pi)
plt.xlabel("# of basis")
plt.ylabel("Subspace angles")
plt.plot(subsp_ang_mx * 180/np.pi)
plt.plot(subsp_ang_pr * 180/np.pi)
# plt.xlabel("Basis functions")
# plt.ylabel("PageRank")
plt.legend(["inc", "mx", "pr"])

In [ ]:
data

In [ ]:
for i in ranked_idx:
    print(ind_mat[i])

In [ ]:
plt.plot(eig_v)
plt.xlabel("Basis functions")
plt.ylabel("PageRank")
plt.plot(eig_v2)
# plt.xlabel("Basis functions")
# plt.ylabel("PageRank")
plt.legend(["PageRank", "Subsampled PageRank"])